In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchWindowException
import getpass
from cryptography.fernet import Fernet
import os, sys, csv, time
from pathlib import Path

# use CONTROL or COMMAND selected by OS
if sys.platform == 'darwin':
    CTRL = Keys.COMMAND
else: CTRL = Keys.CONTROL

desktop = os.path.expanduser("~/Desktop")
setup_path = os.path.join(desktop, 'Deneme')

In [2]:
def open_driver():
    if not os.path.exists(setup_path):
        raise 'SETUP PATH IS NOT VALID OR INACCESSIBLE'
    
    root = os.path.join(setup_path, 'Dersler')
    download_dir = os.path.join(root, '.Downloads')
    
    if not os.path.exists(download_dir):
        os.makedirs(download_dir)
    if not os.path.exists(root):
        os.makedirs(root)

    os.chdir(root)
    
    options = webdriver.ChromeOptions()
    prefs = {"download.default_directory" : download_dir}
    options.add_experimental_option('prefs', prefs)
    
    # Driver
    chrome_driver_path = 'chromedriver'
    driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)

    # Login Website
    driver.get('https://girisv3.itu.edu.tr')
    return driver, root, download_dir

def login(driver):
    # Login
    f = open('.loginfo')
    username = f.readline().strip()
    enc_password = f.readline().strip()
    
    # Find spaces to fill
    user_xpath = '//*[@id="ContentPlaceHolder1_tbUserName"]'
    password_xpath = '//*[@id="ContentPlaceHolder1_tbPassword"]'
    user_place = driver.find_element(By.XPATH, user_xpath)
    password_place = driver.find_element(By.XPATH, password_xpath)
    
    # Fill the spaces
    user_place.send_keys(username)
    fernet = Fernet(bytes(os.environ.get('NINOVA_CREDENTIAL'), 'utf-8'))
    password = fernet.decrypt(bytes(enc_password, 'utf-8')).decode()
    password_place.send_keys(password)
    driver.find_element(By.XPATH, '//*[@id="ContentPlaceHolder1_btnLogin"]').click()
    
    # Go to Ninova
    driver.get("https://ninova.itu.edu.tr/Kampus1")
    
    if 'girisv3' in driver.current_url:
        return False
    
    return True

def login_check(setup_path, driver):
    # save login informations
    loginfo_path = os.path.join(root, '.loginfo')
    if not os.path.exists(loginfo_path) or (os.path.exists(loginfo_path) and os.path.getsize(loginfo_path) == 0):
        loginfo = open(loginfo_path, 'w')
        username = input('Username:')
        password = getpass.getpass()

        # Hash password
        key = Fernet.generate_key()
        os.environ['NINOVA_CREDENTIAL'] = str(key, encoding='utf-8')
        fernet = Fernet(key)
        enc_password = fernet.encrypt(password.encode())
        loginfo.write(username + '\n' + str(enc_password, encoding='utf-8'))
        loginfo.close()

    # Validate login information
    if not login(driver):
        os.remove(loginfo_path)
        raise 'Username or password is incorrect.'
        
    return root, download_dir

def open_course_websites(driver):
    # Open all course websites in a new tab
    lessons = driver.find_element(By.XPATH, '//*[@id="aspnetForm"]/div[3]/div[3]/div[2]/div/div[1]/ul')
    lessons_links = lessons.find_elements(By.TAG_NAME, 'a')
    
    actions = ActionChains(driver)
    for i in lessons_links:
        actions.key_down(CTRL).click(i).key_up(CTRL)
        actions.perform()
    
    driver.switch_to.window(driver.window_handles[0])
    driver.close()
    
def open_class_course_files(driver):
    actions = ActionChains(driver)
    for i in driver.window_handles:
        driver.switch_to.window(i)
        for i in driver.find_elements(By.CLASS_NAME, 'panoElemani'):
            pano_element = i.find_element(By.CSS_SELECTOR, 'h2 > a')
            if pano_element.text in ['Ders Dosyaları', 'Sınıf Dosyaları', 'Class Files', 'Course Files']:
                actions.key_down(CTRL).click(pano_element).key_up(CTRL)
                actions.perform()
        driver.close()
        
def get_local_path(driver, root):
    path_list = driver.find_element(By.CLASS_NAME, 'ic').text.split('/')
    path = os.path.join(root, *list(map(str.strip, path_list))[2:])

    if not os.path.exists(path):
        os.makedirs(path)
    return path

In [3]:
driver, root, download_dir = open_driver()
login_check(setup_path, driver)
open_course_websites(driver)
open_class_course_files(driver)

/var/folders/v0/w3_rg1yn73q9cjs3q6kf4v400000gn/T/ipykernel_23896/938424831.py:21: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_path, chrome_options=options)


TypeError: encoding without a string argument

In [6]:
os.environ['NINOVA_CREDENTIAL'] = 'Emircan'

In [4]:
os.environ.get('NINOVA_CREDENTIAL')

'gV7o7iTriA5oBnPNf_JHLHIPD35Urlv8s5tBpvUSN9M='

In [6]:
archive = open('.archive.csv', 'a+', newline='')
fieldnames = ['path', 'date']
writer = csv.DictWriter(archive, fieldnames=fieldnames)
archive.seek(0)
if archive.read() == '':    
    writer.writeheader()
    archive.close()
    
    archive = open('.archive.csv', 'a+', newline='')
    writer = csv.DictWriter(archive, fieldnames=fieldnames) 

reader = csv.DictReader(archive, fieldnames=fieldnames)

In [7]:
tbody_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/table[2]/tbody'
for page in driver.window_handles:
    driver.switch_to.window(page)
    files_table = driver.find_element(By.XPATH, tbody_xpath)
    actions = ActionChains(driver)
    
    try:
        cells = files_table.find_elements(By.CSS_SELECTOR, 'tr')
        for row in cells:
            try:
                path = get_local_path(driver, root)
                link = row.find_element(By.CSS_SELECTOR, 'td > a')
                date = row.find_element(By.CSS_SELECTOR, ':nth-child(3)').text
                
                if not link.text in dict(reader).values():
                    actions.key_down(CTRL).click(link).key_up(CTRL)
                    actions.perform()
            
                # Wait until file is downloaded
                
                download_dir_content = os.listdir(download_dir)
                if '.DS_Store' in download_dir_content: download_dir_content.remove('.DS_Store')

                while not download_dir_content:
                    time.sleep(1)
                    download_dir_content = os.listdir(download_dir)
                    if '.DS_Store' in download_dir_content: download_dir_content.remove('.DS_Store')

                if len(download_dir_content) == 1:
                    downloading_file = download_dir_content[0]
                    while downloading_file.endswith('.crdownload'):
                        download_dir_content = os.listdir(download_dir)
                        if '.DS_Store' in download_dir_content: download_dir_content.remove('.DS_Store')
                        if len(download_dir_content) == 1:
                            downloading_file = download_dir_content[0]
                        else:
                            raise 'There are many or no files downloading.'

                        time.sleep(0.3) # Time can be estimated in future work (file size / internet speed)

                file_path = os.path.join(download_dir, downloading_file)
                os.system(f'mv "{file_path}" "{path}"')
                writer.writerow({'path':os.path.join(path, link.text.replace(',', ' ')), 'date':date.replace(',', ' ')})
            
            except NoSuchElementException:
                pass
            
    except StaleElementReferenceException or NoSuchElementException:
        pass

MaxRetryError: HTTPConnectionPool(host='localhost', port=53134): Max retries exceeded with url: /session/9865095783e0ef261fce7386bb2c5248/window/handles (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x108568400>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [9]:
archive.close()

In [35]:
files_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/table[2]/tbody'
element = driver.find_element(By.XPATH, files_xpath)

cell = element.find_elements(by=By.CSS_SELECTOR, value='tr')

try:
    link = cell[1].find_element(by=By.CSS_SELECTOR, value='td > a')
    element = actions.context_click(link).send_keys(Keys.DOWN).send_keys(Keys.DOWN)
    actions.perform()
    print(link.text)
    
except NoSuchWindowException:
    pass

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.177)
Stacktrace:
0   chromedriver                        0x0000000102f911c0 chromedriver + 4248000
1   chromedriver                        0x0000000102f11dc0 chromedriver + 3726784
2   chromedriver                        0x0000000102bc6ec4 chromedriver + 274116
3   chromedriver                        0x0000000102ba15f4 chromedriver + 120308
4   chromedriver                        0x0000000102c2af8c chromedriver + 683916
5   chromedriver                        0x0000000102c3d864 chromedriver + 759908
6   chromedriver                        0x0000000102bf5200 chromedriver + 463360
7   chromedriver                        0x0000000102bf6318 chromedriver + 467736
8   chromedriver                        0x0000000102f5f060 chromedriver + 4042848
9   chromedriver                        0x0000000102f638a4 chromedriver + 4061348
10  chromedriver  

In [23]:
for page in driver.window_handles:
    driver.switch_to.window(page)
    files_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/table[2]/tbody'
    element = driver.find_element(By.XPATH, files_xpath)

    for cell in element.find_elements(by=By.CSS_SELECTOR, value='tr > td'):
        try: 
            link = cell.find_element(by=By.CSS_SELECTOR, value='a')
            actions.context_click(link).key_down()
            actions.perform()
            time.sleep(1)
            print(link.text)

        except NoSuchElementException as exc:
            continue

MaxRetryError: HTTPConnectionPool(host='localhost', port=51644): Max retries exceeded with url: /session/88a1ba6b46a39e6184f952fb07a31ae5/window/handles (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x110233f70>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [73]:
for page in driver.window_handles:
    files_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/table[2]/tbody'
    element = driver.find_element(By.XPATH, files_xpath)

    for cell in element.find_elements(by=By.CSS_SELECTOR, value='tr'):
        try: 
            link = cell.find_element(by=By.CSS_SELECTOR, value='td > a')
            actions.key_down(CTRL).click(link).key_up(CTRL)
            actions.perform()

        except NoSuchElementException as exc:
            continue
    page.close()

In [21]:
for page in driver.window_handles:
    files_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/table[2]/tbody'
    element = driver.find_element(By.XPATH, files_xpath)

    for cell in element.find_elements(by=By.CSS_SELECTOR, value='tr'):
        try: 
            link = cell.find_element(by=By.CSS_SELECTOR, value='td > a')
            print(link.text)

        except NoSuchElementException as exc:
            continue

In [27]:
if not "Dersler" in os.listdir(os.getcwd()):
    os.mkdir('Dersler')
os.chdir('Dersler')
lessons_root = os.getcwd()

In [4]:
downloads_path = str(Path.home() / "Downloads")

In [8]:
for root, dirs, files in os.walk(downloads_path):
    dirs = [d for d in dirs if not d[0] == '.']
    # os.remove()
    files = [f for f in files if not f[0] == '.']
    for file in files:
        print(file)

In [9]:
import glob
for file in glob.glob(downloads_path):
    print(file)

/Users/emircan/Downloads


In [17]:
for i in driver.window_handles:
    driver.switch_to.window(i)
    path_xpath = '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/div[1]'
    print(driver.find_element(By.XPATH, path_xpath).text.strip('/'))

In [20]:
driver.switch_to.window(driver.window_handles[8])
course_name = driver.find_element(By.XPATH, '//*[@id="aspnetForm"]/div[3]/div[2]/div[1]/a[3]').text

In [21]:
if not course_name in os.listdir():
    os.mkdir(course_name)
os.chdir(course_name)

In [22]:
path = driver.find_element(By.XPATH, '//*[@id="aspnetForm"]/div[3]/div[3]/div[3]/div/div[2]/div[1]').text[1:]

In [23]:
# distinction for class, course files
class_or_course = driver.find_element(By.XPATH, '//*[@id="ctl00_pnlHeader"]/h1').text

In [24]:
# create a new folder to clone
if not class_or_course in os.listdir():
    os.mkdir(class_or_course)
os.chdir(class_or_course)

In [25]:
lessons_root = 'Dersler'
if os.path.exists(path):
    os.chdir(path)
    os.replace('Downloads', path)

In [26]:
# open all files/folders in a new tab
links = driver.find_elements(By.CSS_SELECTOR, '#aspnetForm > div.tasiyici > div.icerik > div.orta > div > div.dosyaSistemi > table.data > tbody tr:not(:first-child) a')
for i in links:
    actions.key_down(ctrl).click(i).key_up(ctrl)
    actions.perform()